In [1]:
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import pandas as pd
import itertools

from scipy.signal import savgol_filter
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline, make_union
import statsmodels.api as sm
from statsmodels.graphics import regressionplots as rplots
from statsmodels.graphics import gofplots 
from statsmodels.nonparametric.kernel_regression import KernelReg

#from sklearn_xarray import Stacker, Select

In [2]:
##function definition
from functions import *
from climada_functions import *
from constants import * 
import loess_jtrive
from Linear_Reg_Diagnostic import Linear_Reg_Diagnostic

In [3]:
def standardize(df):
    # anomalies
    andf = df - df.mean()
    # standardized
    std_df = andf/df.std()
    return std_df

## Select and load data

In [4]:
## select data
#select variable (cmip6 naming)
selvar = 'sfcWindmax'
pathinvar = pathcmip6+'sfcWindmax/'

#preprocessing 
gst_fact = 1.67
qt = 0.98
cut=5E5
min_lat=30
max_lat=75
min_lon=-30
max_lon=30

##climada constants
haz_type = 'WS'
haz_id = 1

## naming
#name base (meteo) variable
metvar = [cmip6vars[selvar]]
spaceres = ["br_rg"] #base resolution regridded
timeres = ["day"]
domain = ["EU"]
season = ["winE"]
scen = ["allscens"]
sep = "_"
lst_bn = metvar+spaceres+timeres+domain+season
basenamemet = sep.join(lst_bn)

#preproc field
processings = ["qt"+str(qt)[-2:]+"pst","cutarea"+format(cut,'.0E').replace("+0",''),"gst1-67"]
basenamemet_proc = make_fn(processings,basenamemet)


# load indices

In [160]:
#get sfcT
sfcT_fn = "diff_all_remote_indices_O20_sp_avg_allmods_historical_ssp585.csv"
#sfcT_fn = "diff_memmean_all_remote_indices_O20_sp_avg_allmods_historical_ssp585.csv"
sfcT = pd.read_csv(pathcirc+sfcT_fn,header=[0],index_col=[0,1]).loc[:,["sfcT"]]

#get remote indices
dind_fn1 = "diff_all_remote_indices_O20_sp_avg_allmods_historical_ssp585.csv"
#dind_fn1 = "diff_memmean_all_remote_indices_O20_sp_avg_allmods_historical_ssp585.csv"

dind_fn2 = "diff_custom_indices_sp_avg_allmods_historical_ssp585.csv"
dind_fn3 = "diff_weather_indices_CU21_sp_avg_allmods_historical_ssp585.csv"
#dind_fn3 = "diff_memmean_weather_indices_CU21_sp_avg_allmods_historical_ssp585.csv"



preds0 = ["pol","trop","strat"]
preds1 = ["pol","trop","strat","NAWH","Nino4-Nino3"]
preds2 = ["baro","TCWV"]
preds3 = ["NAO","EA","EAWR","SCA"]
dind_df1 = pd.read_csv(pathcirc+dind_fn1,header=[0],index_col=[0,1]).iloc[:,1:]
dind_df2 = pd.read_csv(pathcirc+dind_fn2,header=[0],index_col=[0,1])
dind_df3 = pd.read_csv(pathcirc+dind_fn3,header=[0],index_col=[0,1])

#concat 
dind_df_all = pd.concat([dind_df1,dind_df3],axis=1)

In [96]:
dind_df_all.loc['CNRM-CM6-1-HR']

,pol,trop,strat,NAWH,Nino4-Nino3,NAO,EA,EAWR,SCA
imem,,,,,,,,,
0,7.974094,7.875693,-2.434483,4.023071,-0.413656,0.111739,4.786972,-0.610051,0.993002
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [97]:
dind_df_all

pol      trop     strat      NAWH  Nino4-Nino3  \
model         imem                                                        
AWI-CM-1-1-MR 0     7.146866  5.490276 -1.182888  2.809386    -0.263903   
              1          NaN       NaN       NaN       NaN          NaN   
              2          NaN       NaN       NaN       NaN          NaN   
BCC-CSM2-MR   0     6.891896  5.334170 -0.816363  2.460681    -0.111383   
              1          NaN       NaN       NaN       NaN          NaN   
...                      ...       ...       ...       ...          ...   
MPI-ESM1-2-LR 1     5.916541  5.257276  0.494406  2.202200     0.030194   
              2     6.345129  5.102040  0.300191  2.355241     0.111310   
KACE-1-0-G    0    -9.602921  7.230911  1.242553  3.258705    -0.182288   
              1     8.068593  7.317720  0.088979  3.375989    -0.182063   
              2     7.401835  7.191004  0.002456  3.276444    -0.120468   

                         NAO        EA      EAWR       SCA  
model         imem                                          
AWI-CM-1-1-MR 0     0.152881  3.437205 -0.544412  0.491510  
              1          NaN       NaN       NaN       NaN  
              2          NaN       NaN       NaN       NaN  
BCC-CSM2-MR   0     0.273082  3.292483 -0.648636  0.263525  
              1          NaN       NaN       NaN       NaN  
...                      ...       ...       ...       ...  
MPI-ESM1-2-LR 1     0.372667  3.410667 -0.132501  0.262932  
              2     0.314178  3.007412 -0.257616  0.408893  
KACE-1-0-G    0     0.612771  4.492766 -0.208132 -0.148585  
              1     0.564277  4.509497 -0.558158  0.192932  
              2     0.474178  4.461972 -0.243214  0.254782  

[87 rows x 9 columns]

## Get target values

In [119]:
#select vars
modlist = modlist_ssp585 + modlist_allscen
modset = "allmods"
sel_impf = 'Em2011'
proctype = 'diff_qt'
simname = 'stacked'
caltype = "AAI_EMDAT_100mn"
imp_metric = 1

regions1 = ['UK','WEU','SEU','NOR','EEU'] #leave empty for EU
regions2 = ['BI','IP','FR','MID','MED','SC','EEU']
regions3 = ['BI','IP','FR','WEU','MED','SC','EEU']

reglist = regions2
reglist = reglist + ["EU"] #add europe
pastname = 'historical'
futname = 'ssp585'
timeres='day'
nmem_max = 3
qt = 0.98
#regaaifn = make_fn(["reg",imp_metric,sel_impf,caltype,modset,futname,pastname],basename=basenamemet_proc,filetype=".csv")
memname_df = pd.read_csv('/home/lseverino/MT/metadata/memnames_ssp585_hist_SWM.csv',header=[0,1],index_col=0)

In [120]:
#intiate df to save result
#initiate df to store results
iterrows = [modlist,range(nmem_max)]
row_idx= pd.MultiIndex.from_product(iterrows,names=["model","imem"])
itercols = [reglist,[pastname,futname]]

col_idx= pd.MultiIndex.from_product(itercols,names=["region","period"])

reg_aai_df = pd.DataFrame(index=row_idx,columns=col_idx)



In [121]:
##construct data arrays for regression stacked

#impact basename
savenameimp = make_fn([sel_impf,caltype,proctype],basenamemet_proc)

for reg in reglist:
    #loop over the models
    for modid, modname in enumerate(modlist):
    
        
        
        #read impact data
        if reg == "EU":
            #past
            impfnp= make_fn(['imp',simname,pastname,modname],savenameimp,filetype='.csv')
            impp = Impact()
            impp = impp.from_csv(pathimp+"impact csv/aggregated/stacked/"+pastname+"/"+impfnp) 
            
            #future
            impfnf= make_fn(['imp',simname,futname,modname],savenameimp,filetype='.csv')
            impf = Impact()
            impf = impf.from_csv(pathimp+"impact csv/aggregated/stacked/"+futname+"/"+impfnf)
        else:
            #past
            impfnp= make_fn(['imp',simname,reg,pastname,modname],savenameimp,filetype='.csv')
            impp = Impact()
            impp = impp.from_csv(pathimp+"impact csv/regional/stacked/"+reg+"/"+pastname+"/"+impfnp) 
            
            #future
            impfnf= make_fn(['imp',simname,reg,futname,modname],savenameimp,filetype='.csv')
            impf = Impact()
            impf = impf.from_csv(pathimp+"impact csv/regional/stacked/"+reg+"/"+futname+"/"+impfnf)
        
        #get impact metrics
        if imp_metric == 'aai_agg':
            aaip = impp.aai_agg
            aaif = impf.aai_agg
        else:
            aaip = impp.calc_freq_curve(return_per=imp_metric).impact
            aaif = impf.calc_freq_curve(return_per=imp_metric).impact
        
        #write into df
        reg_aai_df.loc[modname,(reg,pastname)] = aaip
        reg_aai_df.loc[modname,(reg,futname)] = aaif
reg_aai_df = reg_aai_df.astype(np.float64)
#reg_aai_df.to_csv(pathcirc+regaaifn)

In [122]:
reg_aai_df

region                        BI                          IP                \
period                historical        ssp585    historical        ssp585   
model         imem                                                           
AWI-CM-1-1-MR 0     1.454260e+09  2.175486e+09  6.308469e+08  4.237061e+08   
              1     1.454260e+09  2.175486e+09  6.308469e+08  4.237061e+08   
              2     1.454260e+09  2.175486e+09  6.308469e+08  4.237061e+08   
BCC-CSM2-MR   0     1.395815e+09  1.708436e+09  5.555894e+08  1.014965e+09   
              1     1.395815e+09  1.708436e+09  5.555894e+08  1.014965e+09   
...                          ...           ...           ...           ...   
MPI-ESM1-2-LR 1     8.864146e+08  9.904528e+08  7.956738e+08  7.063749e+08   
              2     8.864146e+08  9.904528e+08  7.956738e+08  7.063749e+08   
KACE-1-0-G    0     1.477206e+09  2.740033e+08  2.980200e+08  2.709362e+08   
              1     1.477206e+09  2.740033e+08  2.980200e+08  2.709362e+08   
              2     1.477206e+09  2.740033e+08  2.980200e+08  2.709362e+08   

region                        FR                          ME                \
period                historical        ssp585    historical        ssp585   
model         imem                                                           
AWI-CM-1-1-MR 0     1.276646e+09  9.145825e+08  1.368706e+09  1.900495e+09   
              1     1.276646e+09  9.145825e+08  1.368706e+09  1.900495e+09   
              2     1.276646e+09  9.145825e+08  1.368706e+09  1.900495e+09   
BCC-CSM2-MR   0     4.169301e+08  8.905483e+08  1.843498e+09  1.071357e+09   
              1     4.169301e+08  8.905483e+08  1.843498e+09  1.071357e+09   
...                          ...           ...           ...           ...   
MPI-ESM1-2-LR 1     7.928048e+08  1.097970e+09  1.784711e+09  1.198972e+09   
              2     7.928048e+08  1.097970e+09  1.784711e+09  1.198972e+09   
KACE-1-0-G    0     1.108265e+09  2.164353e+08  1.705979e+09  1.642334e+08   
              1     1.108265e+09  2.164353e+08  1.705979e+09  1.642334e+08   
              2     1.108265e+09  2.164353e+08  1.705979e+09  1.642334e+08   

region                        MD                          SC                \
period                historical        ssp585    historical        ssp585   
model         imem                                                           
AWI-CM-1-1-MR 0     3.615033e+08  4.316476e+08  3.185331e+08  4.468518e+08   
              1     3.615033e+08  4.316476e+08  3.185331e+08  4.468518e+08   
              2     3.615033e+08  4.316476e+08  3.185331e+08  4.468518e+08   
BCC-CSM2-MR   0     5.053532e+08  7.807520e+08  6.641809e+08  7.301368e+08   
              1     5.053532e+08  7.807520e+08  6.641809e+08  7.301368e+08   
...                          ...           ...           ...           ...   
MPI-ESM1-2-LR 1     3.728698e+08  3.805812e+08  3.305062e+08  3.333147e+08   
              2     3.728698e+08  3.805812e+08  3.305062e+08  3.333147e+08   
KACE-1-0-G    0     4.428406e+08  2.244988e+08  3.346855e+08  4.031230e+08   
              1     4.428406e+08  2.244988e+08  3.346855e+08  4.031230e+08   
              2     4.428406e+08  2.244988e+08  3.346855e+08  4.031230e+08   

region                        EA                          EU                
period                historical        ssp585    historical        ssp585  
model         imem                                                          
AWI-CM-1-1-MR 0     2.695007e+08  4.284628e+08  2.504342e+09  3.944755e+09  
              1     2.695007e+08  4.284628e+08  2.504342e+09  3.944755e+09  
              2     2.695007e+08  4.284628e+08  2.504342e+09  3.944755e+09  
BCC-CSM2-MR   0     1.411909e+08  3.489184e+08  3.894819e+09  3.163259e+09  
              1     1.411909e+08  3.489184e+08  3.894819e+09  3.163259e+09  
...                          ...           ...           ...           ...  
MPI-ESM1-2-LR 1     5.077198

In [10]:
## get model res
#model res df
res_df = pd.DataFrame(columns=["Res"],index=modlist)

for modid, modname in enumerate(modlist):
    fn = make_fn([modname],basenamemet,filetype=".nc")
    ncdf = xr.open_dataset(pathinvar+fn)
    ncdfw = ncdf[[pastname,futname]]
    
    #get resolutiom
    latres, lonres = get_lat_lon_res(ncdfw)
    meanres = np.sqrt(latres**2 + lonres**2)
    res_df.loc[modname,"Res"] = meanres
    res_df.loc[modname] =  meanres
res_df = res_df.astype(np.float64)   

# Multiple Linear Regression

In [161]:
##select data

#models
modlist = modlist_ssp585 + modlist_allscen
#modlist = modlist_1cen
#remove outliers
modlist.remove('CNRM-CM6-1-HR')
#modlist.remove('GISS-E2-1-G')
#modlist.remove('MRI-ESM2-0')
#modlist.remove('INM-CM5-0')
#modlist.remove('KACE-1-0-G')
#modlist.remove('MIROC-ES2L')
#modlist.remove('BCC-CSM2-MR')
#modlist.remove('EC-Earth3-Veg')
#modlist.remove('EC-Earth3-Veg-LR')


#impact data
modset = "allmods"
reglist = reglist
sel_impf = 'Em2011'
caltype = "AAI_EMDAT_100mn"
#imp_metric = "1 yr rp"
pastname = 'historical'
futname = 'ssp585'
timeres='day'
#nmems = 3
#qt = 0.98

#predictors
#preds = ["pol","trop","strat","NAWH","Nino4-Nino3"] #predictors
#preds = ["baro","TCWV"] #predictors
preds = ["NAO","EA","EAWR","SCA"]

#regression model
#specify model
normresp = True #normalized by global warming
memmean = True
logresp = False
addTs = False
addres = False



In [162]:
#remove first member of KACE because outlier
dind_df_all_rm = dind_df_all.drop(('KACE-1-0-G',0),axis=0)
reg_aai_df_rm = reg_aai_df.drop(('KACE-1-0-G',0),axis=0)
sfcT_rm = sfcT.drop(('KACE-1-0-G',0),axis=0)
#dind_df_all_rm = dind_df_all.copy()
#reg_aai_df_rm = reg_aai_df.copy()
#sfcT_rm = sfcT.copy()

In [163]:
##prepare predictors
#select df
sel_dind_memsep_df = dind_df_all_rm.copy().loc[modlist,preds]
sel_sfcT_memsep = sfcT_rm.loc[modlist]

#memmean 
sel_dind_df = sel_dind_memsep_df.groupby("model").mean()
sel_sfcT = sel_sfcT_memsep.groupby("model").mean()

#sel_dind_df = sel_dind_memsep_df.copy()
#sel_sfcT = sel_sfcT_memsep.copy()

#normalize by sfcT
sel_dind_df.loc[:,preds] = sel_dind_df.loc[:,preds] / sel_sfcT.values

# anomalies
an_dind_df = sel_dind_df.copy()
an_dind_df = sel_dind_df.loc[:,preds]-sel_dind_df.loc[:,preds].mean()

# standardized
std_an_dind_df = an_dind_df.copy()
std_an_dind_df.loc[:,preds] = an_dind_df/sel_dind_df.std()

In [164]:
#process target var
#memmean
reg_aai_df_ss = reg_aai_df_rm.groupby("model",axis=0).mean()
if logresp:
    reg_aai_df_ss = np.log(reg_aai_df_ss)

In [165]:
## reindex to ensure data alignement
reg_aai_df_ss = reg_aai_df_ss.reindex(modlist)
ind_df = std_an_dind_df.reindex(modlist)
Ts = sel_sfcT.reindex(modlist)

# Variable selection

In [17]:
import statsmodels.formula.api as smf


def forward_selected(data, response):
    """Linear model designed by forward selection.

    Parameters:
    -----------
    data : pandas DataFrame with all possible predictors and response

    response: string, name of response column in data

    Returns:
    --------
    model: an "optimal" fitted statsmodels linear model
           with an intercept
           selected by forward selection
           evaluated by adjusted R-squared
    """
    remaining = set(data.columns)
    remaining.remove(response)
    y = data[response]
    selected = []
    current_score, best_new_score = 0.0, 0.0
    while remaining and current_score == best_new_score:
        scores_with_candidates = []
        for candidate in remaining:
            formula = "{} ~ {} + 1".format(response,
                                           ' + '.join(selected + [candidate]))
            #data_sel = data[selected + [candidate].add_constant()
            X = sm.add_constant(data[selected + [candidate]])
            fit = sm.RLM(y.values,X, M=sm.robust.norms.TukeyBiweight()).fit() # describe and fit model
            yfit = fit.fittedvalues
            rsq = comp_rsq(yfit.values,y.values)
            adj_rsq = comp_adj_rsq(yfit.values,y.values,X)
            score = adj_rsq
            #score = smf.ols(formula, data).fit().rsquared_adj
            scores_with_candidates.append((score, candidate))
        scores_with_candidates.sort()
        best_new_score, best_candidate = scores_with_candidates.pop()
        if current_score < best_new_score:
            remaining.remove(best_candidate)
            selected.append(best_candidate)
            current_score = best_new_score
    formula = "{} ~ {} + 1".format(response,
                                   ' + '.join(selected))
    #model = smf.ols(formula, data).fit()
    Xsel = sm.add_constant(data[selected])
    model = sm.RLM(y.values,Xsel, M=sm.robust.norms.TukeyBiweight()).fit() # describe and fit model
    yfit = model.fittedvalues
    rsq = comp_rsq(yfit.values,y.values)
    adj_rsq = comp_adj_rsq(yfit.values,y.values,Xsel)
    best_score = adj_rsq

    return model, best_score

In [18]:
import pandas as pd
import statsmodels.api as sm

def forward_regression(X, y,
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    initial_list = []
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.RLM(y, sm.add_constant(pd.DataFrame(X[included])), M=sm.robust.norms.TukeyBiweight()).fit()
            print(model.pvalues)
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add   with p-value '.format(best_feature, best_pval))

        if not changed:
            break

    return included

def backward_regression(X, y,
                           initial_list=[], 
                           threshold_in=0.01, 
                           threshold_out = 0.05, 
                           verbose=True):
    included=list(X.columns)
    while True:
        changed=False
        model = sm.RLM(y, sm.add_constant(pd.DataFrame(X[included])), M=sm.robust.norms.TukeyBiweight()).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop  with p-value '.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [19]:
import pandas as pd
import numpy as np
import statsmodels.api as sm


def stepwise_selection(X, y, 
                       initial_list=[], 
                       threshold_in=0.01, 
                       threshold_out = 0.05, 
                       verbose=True):
    """ Perform a forward-backward feature selection 
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features 
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.argmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.argmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included


In [20]:
#mlr for each region using statsmodel
X = ind_df.copy()


#X = sm.add_constant(X) # adding a constant
best_mod_dict = {}
preds_f_dict = {}
preds_b_dict = {}
#sum_tables_best = {}
#rsq_dict_best = {} 
#rsq_adj_dict_best = {}
for reg in reglist:
    #select data
    aai_reg = reg_aai_df_ss.loc[:,reg]
    ##diff
    #y = aai_reg.loc[:,futname] - aai_reg.loc[:,pastname]
    
    #ratio
    #y = aai_reg.loc[:,futname] / aai_reg.loc[:,pastname]
    
    #diff ratio
    y = 100*(aai_reg.loc[:,futname]-aai_reg.loc[:,pastname]) / aai_reg.loc[:,pastname]

    if normresp:
        y = y.divide(Ts.values.flatten())
    
    best_mod_dict[reg] = stepwise_selection(X, y,threshold_in=0.05, threshold_out = 0.1)

    #best_mod_dict[reg] = model
    #rsq_adj_dict_best[reg] = best_score
    

/tmp/ipykernel_5946/846559375.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)
/tmp/ipykernel_5946/846559375.py:29: FutureWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  new_pval = pd.Series(index=excluded)


Add                               4 with p-value 0.000396014


KeyError: "None of [Int64Index([4], dtype='int64')] are in the [columns]"

## Stepwise regression

In [ ]:
import numpy as np
import warnings
import os
import statsmodels.formula.api as smf
import pandas as pd
import functools
import re
warnings.filterwarnings('ignore')


class stepwise:

    def __init__(self,step,fit_intercept):
        self.step = step
        self.fit_intercept = fit_intercept

    def reduce_concat(self,x, sep=""):
        return functools.reduce(lambda x, y: str(x) + sep + str(y), x)

    def fit(self,data,null_formula,full_formula,response):

        """Linear model designed by forward selection.
        Parameters:
        -----------
        data : pandas DataFrame with all possible predictors and response
        response: string, name of response column in data
        Returns:
        --------
        model: an "optimal" fitted statsmodels linear model
               with an intercept
               selected by forward selection
               evaluated by aic
        """

        null_temp        = re.split('~',null_formula)
        null_predic_com  = null_temp[1].split('+')
        null_predic      = null_predic_com[1:len(null_predic_com)]
        full_temp        = re.split('~',full_formula)
        full_predic_com  = full_temp[1].split('+')
        full_predic      = full_predic_com[1:len(full_predic_com)]
        indices          = [i for i,id in enumerate(full_predic) if id not in null_predic]
        domain           = [full_predic[i] for i in indices]
        start            = set(null_predic)
        remaining        = set(domain)
        selected         = null_predic
        current_score, best_new_score = 0, 0
        score_selected   = []
        variable_added   = []
        y = data[response]
        #selected.remove('')
        while (remaining and current_score == best_new_score and self.step >0):
            scores_with_candidates = []
            for candidate in remaining:
                formula = "{} ~ {}".format(response,' + '.join(selected + [candidate]))
                if self.fit_intercept == 0:
                    formula = formula + "-1"
                X = sm.add_constant(data[selected + [candidate]])
                fit = sm.RLM(y.values,X, M=sm.robust.norms.TukeyBiweight()).fit()
                yfit = fit.fittedvalues
                rsq = comp_rsq(yfit.values,y.values)
                adj_rsq = comp_adj_rsq(yfit.values,y.values,X)
                score = adj_rsq
                #score = -smf.ols(formula, data).fit().rsquared_adj
                scores_with_candidates.append((score, candidate))
            scores_with_candidates.sort()
            best_new_score, best_candidate = scores_with_candidates.pop()
            if current_score < best_new_score:
                remaining.remove(best_candidate)
                selected.append(best_candidate)
                score_selected.append(best_new_score)
                variable_added.append(best_candidate)
                current_score = best_new_score
            self.step=self.step-1
        formula = "{} ~ {}".format(response,' + '.join(selected))
        if self.fit_intercept == 0:
            formula = formula + "-1"
        #model = smf.ols(formula, data).fit()
        
        Xsel = sm.add_constant(data[selected])
        model = sm.RLM(y.values,Xsel, M=sm.robust.norms.TukeyBiweight()).fit() # describe and fit model
        yfit = model.fittedvalues
        rsq = comp_rsq(yfit.values,y.values)
        adj_rsq = comp_adj_rsq(yfit.values,y.values,Xsel)
        best_score = adj_rsq
        return model, best_score, selected

In [ ]:
#forward var selection
best_mod_dict = {}
sum_tables_best = {}
best_preds_dict = {} 
best_score_dict = {}
X = ind_df.copy()
for reg in reglist:
    #select data
    aai_reg = reg_aai_df_ss.loc[:,reg]
    ##diff
    #y = aai_reg.loc[:,futname] - aai_reg.loc[:,pastname]
    
    #ratio
    #y = aai_reg.loc[:,futname] / aai_reg.loc[:,pastname]
    
    #diff ratio
    y = 100*(aai_reg.loc[:,futname]-aai_reg.loc[:,pastname]) / aai_reg.loc[:,pastname]

    if normresp:
        y = y.divide(Ts.values.flatten())
    model_df = pd.concat([y,X],axis=1).astype(np.float64)
    if 'Nino4-Nino3' in preds:
        preds.remove('Nino4-Nino3')
        preds.append('Nino43')
    
    model_df.columns = ['y']+ preds
    full_form = 'y~1+'+'+'.join(preds)
    null_form = 'y~1'
    best_mod = stepwise(20,True)
    best_mod, best_score, best_preds = best_mod.fit(model_df,null_form,full_form,'y')
    
    best_mod_dict[reg] = best_mod
    best_score_dict[reg] = best_score
    best_preds_dict[reg] = best_preds
    

## Regsubset

In [166]:
def processSubset(feature_set):
    # Fit model on feature_set and calculate RSS
    Xi = sm.add_constant(X[list(feature_set)].copy())
    #model = sm.OLS(y,Xi)
    model = sm.RLM(y,Xi, M=sm.robust.norms.TukeyBiweight())
    #model = sm.OLS(y,Xi)
    regr = model.fit()
    #yfit = regr.fittedvalues
    rsq = comp_rsq(regr,y.values)
    adj_rsq = comp_adj_rsq(regr,y.values,Xi)
    #rsq = regr.rsquared
    #adj_rsq = regr.rsquared_adj
    RSS = ((regr.predict(Xi) - y) ** 2).sum()
    
    return {"model":regr, "RSS":RSS , "adj-Rsq":adj_rsq, "Rsq":rsq}

def getBest(k):
    
    start_time = timer()
    
    results = []
    
    for combo in itertools.combinations(X.columns, k):
        results.append(processSubset(combo))
    
    # Wrap everything up in a nice dataframe
    models = pd.DataFrame(results)
    
    # Choose the model with the highest RSS
    best_model = models.iloc[models['adj-Rsq'].argmax()]
    
    time_delta_past = timer() - start_time
    print("Processed", models.shape[0], "models on", k, "predictors in", time_delta_past, "seconds.")
    
    # Return the best model, along with some other useful information about the model
    return best_model


In [167]:
#mlr for each region using statsmodel
best_mod_dict = {}
best_preds_dict = {} 
best_score_dict = {}
rsq_dict = {}
X = ind_df.copy()
npreds_max = X.shape[1]
#metric = 'adj-Rsq'
for reg in reglist:
    #select data
    aai_reg = reg_aai_df_ss.loc[:,reg]
    ##diff
    #y = aai_reg.loc[:,futname] - aai_reg.loc[:,pastname]
    
    #ratio
    #y = aai_reg.loc[:,futname] / aai_reg.loc[:,pastname]
    
    #diff ratio
    y = 100*(aai_reg.loc[:,futname]-aai_reg.loc[:,pastname]) / aai_reg.loc[:,pastname]

    if normresp:
        y = y.divide(Ts.values.flatten())
    
    
    models_best = pd.DataFrame(columns=["RSS","Rsq", "adj-Rsq","model"])    
    start_time = timer()
    
    for i in range(1,npreds_max+1):
        models_best.loc[i] = getBest(i)
    models_best[['RSS','Rsq','adj-Rsq']] = models_best[['RSS','Rsq','adj-Rsq']].astype(np.float64)
    
    time_delta_past = timer() - start_time    
    print("Total elapsed time:", time_delta_past, "seconds.")
    
    best_mod = models_best.iloc[models_best['adj-Rsq'].argmax()]
    best_mod_dict[reg] = best_mod.loc['model']
    best_score_dict[reg] = best_mod.loc['adj-Rsq']
    rsq_dict[reg] = best_mod.loc['Rsq']
    best_preds_dict[reg] = best_mod.loc['model'].model.data.param_names
    

Processed 4 models on 1 predictors in 0.02854863926768303 seconds.
Processed 6 models on 2 predictors in 0.05027442052960396 seconds.
Processed 4 models on 3 predictors in 0.03515142574906349 seconds.
Processed 1 models on 4 predictors in 0.010191570967435837 seconds.
Total elapsed time: 0.13146261125802994 seconds.
Processed 4 models on 1 predictors in 0.03318721055984497 seconds.
Processed 6 models on 2 predictors in 0.05768314376473427 seconds.
Processed 4 models on 3 predictors in 0.038498878479003906 seconds.
Processed 1 models on 4 predictors in 0.009662248194217682 seconds.
Total elapsed time: 0.14695492386817932 seconds.
Processed 4 models on 1 predictors in 0.028291910886764526 seconds.
Processed 6 models on 2 predictors in 0.05082374066114426 seconds.
Processed 4 models on 3 predictors in 0.03920193016529083 seconds.
Processed 1 models on 4 predictors in 0.012904170900583267 seconds.
Total elapsed time: 0.1390070840716362 seconds.
Processed 4 models on 1 predictors in 0.02697

In [168]:
## construct table robust mlr
row_idx = ["const"]+preds+["Rsq","adj-Rsq"]
best_mod_sum_df_rob = pd.DataFrame(index=row_idx,columns=reglist)
best_mod_pval_df_rob = pd.DataFrame(index=["const"]+preds,columns=reglist)

for reg in reglist:
    fitreg = best_mod_dict[reg]
    #coefs and pvals
    coefs = fitreg.params
    pvals = fitreg.pvalues
    for varname, coef in coefs.items():
        best_mod_sum_df_rob.loc[varname,reg] = coef
        best_mod_pval_df_rob.loc[varname,reg] = pvals[varname]
    
    
    best_mod_sum_df_rob.loc["Rsq",reg] = rsq_dict[reg]
    best_mod_sum_df_rob.loc["adj-Rsq",reg] = best_score_dict[reg]
    #best_mod_sum_df_rob.loc["Rsq",reg] = best_score_dict[reg]

    

In [169]:
def style_negative(v, props=''):
    return props if v < 0 else None

def highlight_max(s, props=''):
    return np.where(s == np.nanmax(s.values), props, '')

def show_significance(s, threshold=0.1):
    if s <= threshold:
        return 'color:{0}; font-weight:bold;textit:--rwrap; textbf:--rwrap;'.format('red')
    else:
        return ''

In [170]:
slice_ = preds
s = best_mod_sum_df_rob.drop("const",axis=0).style.apply(lambda x: best_mod_pval_df_rob.drop("const",axis=0).applymap(show_significance), axis=None).format('{:.2f}',na_rep='-')
s

,BI,IP,FR,MID,MED,SC,EEU,EU
NAO,5.80,2.15,6.89,5.35,4.31,4.38,7.83,4.31
EA,-,-,-2.56,-3.35,-3.58,-2.41,-4.76,-
EAWR,-,-2.35,-2.64,-2.95,-3.66,-,-3.22,-4.29
SCA,-,-,-,-,-,-,2.08,-
Rsq,0.02,0.30,0.35,0.21,0.38,0.15,0.40,0.32
adj-Rsq,-0.02,0.25,0.26,0.11,0.30,0.08,0.29,0.27


In [118]:
print(s.to_latex())

\begin{tabular}{lllllllll}
 & BI & IP & FR & WEU & MED & SC & EEU & EU \\
pol & - & \colorred \font-weightbold \textit{\textbf{-2.03}} & -3.44 & -1.52 & -1.96 & \colorred \font-weightbold \textit{\textbf{-5.04}} & - & - \\
trop & - & 1.48 & - & - & - & \colorred \font-weightbold \textit{\textbf{-3.29}} & - & - \\
strat & - & \colorred \font-weightbold \textit{\textbf{-3.22}} & -1.94 & - & \colorred \font-weightbold \textit{\textbf{-3.72}} & -1.24 & - & - \\
NAWH & -3.36 & -1.46 & \colorred \font-weightbold \textit{\textbf{-3.63}} & - & \colorred \font-weightbold \textit{\textbf{-2.65}} & - & - & -3.03 \\
Nino4-Nino3 & - & \colorred \font-weightbold \textit{\textbf{3.26}} & 3.34 & - & \colorred \font-weightbold \textit{\textbf{4.19}} & 2.64 & 1.72 & - \\
Rsq & 0.03 & 0.50 & 0.23 & 0.02 & 0.32 & 0.29 & 0.03 & 0.06 \\
adj-Rsq & -0.01 & 0.38 & 0.10 & -0.02 & 0.20 & 0.17 & -0.01 & 0.02 \\
\end{tabular}

